### MOVIES CLUSTERING

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import TruncatedSVD

h:\Netflix_Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## FILTER ONLY MOVIES

In [4]:
netflix_data_cleaned = pd.read_csv(r"H:\Netflix_Project\data\processed\cleaned_netflix_data.csv")

In [12]:
netflix_data_cleaned.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,combined_text,cluster
0,s1,TV Show,3%,Unknown,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,2020-08-14,2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...,2020.0,In a future where the elite inhabit an island ...,2
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,2016-12-23,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...,2016.0,After a devastating earthquake hits Mexico Cit...,3
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,2018-12-20,2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow...",2018.0,"When an army recruit is found dead, his fellow...",1
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,2017-11-16,2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...",2017.0,"In a postapocalyptic world, rag-doll robots hi...",3
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,2020-01-01,2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,2020.0,A brilliant group of students become card-coun...,0


In [5]:
movies_data = netflix_data_cleaned[netflix_data_cleaned['type'] == "Movie"]

tv_data = netflix_data_cleaned[netflix_data_cleaned["type"] == "TV Show"]

In [6]:
netflix_data_cleaned["combined_text"] = netflix_data_cleaned["description"] + " " + netflix_data_cleaned["listed_in"]

In [7]:
movies_data.shape

(5377, 15)

In [8]:
tv_data.shape

(2312, 15)

In [ ]:
## CREATE TEXT FEATURE

In [13]:
movies_data_cleaned["combined_text"] = (movies_data_cleaned["description"] + " " + movies_data_cleaned["listed_in"])

NameError: name 'movies_data_cleaned' is not defined

In [14]:
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=10000,
    ngram_range=(1,2)
)

X_movies = tfidf.fit_transform(movies_data["combined_text"])

In [15]:
svd = TruncatedSVD(n_components=100, random_state=42)
X_movies_reduced = svd.fit_transform(X_movies)

In [16]:
for k in range(2, 12):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_movies_reduced)
    score = silhouette_score(X_movies_reduced, labels)
    print(f"K={k}, Score={score}")

K=2, Score=0.03661627514224159
K=3, Score=0.046783282310198994
K=4, Score=0.05686082772871886
K=5, Score=0.0645060077319708
K=6, Score=0.08122251704535341
K=7, Score=0.08949917830517004
K=8, Score=0.08384719378773638
K=9, Score=0.08338917199528127
K=10, Score=0.09391537273571207
K=11, Score=0.1058318733971459


In [ ]:
kmeans = KMeans(n_clusters=11, random_state=42)
movies_data["cluster"] = kmeans.fit_predict(X_movies_reduced)

In [ ]:
for i in range(movies_data["cluster"].nunique()):
    print(f"\nCluster {i}")
    print(movies_data[movies_data["cluster"] == i]["listed_in"].value_counts().head(5))

In [ ]:
pd.crosstab(movies_data["cluster"], movies_data["country"]).head()

In [ ]:
pd.crosstab(movies_data["cluster"], movies_data["rating"])

In [ ]:
pd.crosstab(movies_data["cluster"], movies_data["year_added"])